In [ ]:
import gpmultipy, scipy, GPy
import numpy as np
import matplotlib.pyplot as plt
import json,copy

In [ ]:
from gpmultipy import dataset
from gpmultipy.model import Model
from gpmultipy.prior import Prior
from gpmultipy.kernel import RBF, White, Addition
from gpmultipy.freeze import Freezer
from gpmultipy.sampler.slice import Slice
from gpmultipy.kernel import Product, Linear

In [ ]:
%matplotlib inline

In [ ]:
def printSample(sample,depth=1):
    keys = sample.keys()
    keys.sort()
    
    for k in keys:
        if type(sample[k]) == dict:
            print "\t".join([""]*depth) + k
            printSample(sample[k],depth=depth+1)
        else:
            print "\t".join([""]*depth) + "%s: %s"%(k,str(sample[k]))

In [ ]:
ds = dataset.DataSet('data/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',effects=['Well','Bio'])
y = np.log2(y)
y = (y-y.mean())/y.std()

In [ ]:
edge = range(101,111) + range(111,191,10) + range(120,191,10) + range(191,201) + range(201,211) + range(211,291,10) + range(220,291,10) + range(291,301)
edge = np.array(edge)
edge

In [ ]:
dist = np.array([min(abs(l-edge)) for l in labels[0]])
position = (dist==0).astype(int)

In [ ]:
xgp = np.zeros((x.shape[0]*y.shape[1], 3))
xgp[:,0] = np.tile(x[:,0], y.shape[1])
xgp[:,1] = np.repeat(position, x.shape[0])
xgp[:,2] = np.repeat(effect.Bio, x.shape[0])

ygp = y.ravel(1)[:,None]

xgp = xgp[::10]
ygp = ygp[::10]

xgp.shape

In [ ]:
kmain = GPy.kern.RBF(2, ARD=True,name='main')
kbio = GPy.kern.RBF(2,ARD=True,name='bio')
k = GPy.kern.Hierarchical([kmain, kbio])
#k = GPy.kern.RBF(1,name='time')*GPy.kern.RBF(1,name='edge',active_dims=[1])

m = GPy.models.GPRegression(xgp,ygp,k)
m

In [ ]:
m.optimize()
m

In [ ]:
plt.bar(range(2), [kmain.variance, kbio.variance], tick_label=['main','bio'])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title("$\sigma$",fontsize=20)
plt.tight_layout()
plt.savefig("figures/kern-variance-withBio.pdf")

In [ ]:
xpred = np.zeros((100,3))
xpred[:50,0] = xpred[50:,0] = np.linspace(x.min(),x.max())
xpred[50:,1] = 1
xpred[:,2] = 1000

mu,cov = m.predict_noiseless(xpred,full_cov=True, kern=kmain)

op = np.zeros((50,100))
for i in range(50):
    op[i,i] = 1
    op[i,i+50] = -1
    
mu = np.dot(op,mu)[:,0]
cov = np.dot(op,np.dot(cov,op.T))

plt.plot(xpred[:50,0],mu)
plt.fill_between(xpred[:50,0],mu+1.96*np.sqrt(cov.diagonal()),mu-1.96*np.sqrt(cov.diagonal()),alpha=.1)
plt.plot([x.min(),x.max()],[0,0],'k',lw=3)

plt.plot(x,y[:,position==0].mean(1) - y[:,position==1].mean(1))

plt.xlabel("time", fontsize=15)
plt.ylabel("log(OD)", fontsize=15)
plt.tight_layout()

plt.savefig("figures/edge-effect-gp-withBio.pdf")

In [ ]:
ds = dataset.DataSet('data/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',effects=['Well'])
y = np.log2(y)

ymean,ystd = y.mean(),y.std()
y = (y-y.mean())/y.std()

In [ ]:
edge = range(101,111) + range(111,191,10) + range(120,191,10) + range(191,201) + range(201,211) + range(211,291,10) + range(220,291,10) + range(291,301)
edge = np.array(edge)
edge

In [ ]:
dist = np.array([min(abs(l-edge)) for l in labels[0]])
position = (dist==0).astype(int)

In [ ]:
yKernel = Addition(White(1,.1),RBF(1,.1))
k1 = RBF(1,1,1)
k2 = RBF(1,1,.1)

p = y.shape[1]
dm = np.ones((2,p))
dm[1,:] = 1-2*position

prior = Prior(x,k1,[0])
prior2 = Prior(x,k2,range(1,dm.shape[0]))

model = Model(x,y,dm)

In [ ]:
ySigmaSlice = Slice('ySigma',
                    lambda x: model.dataLikelihood(yKernel,k1_sigma=x),
                    lambda x: scipy.stats.uniform(1e-6,1e2).logpdf(x),
                    .2,5,logspace=True)

ySigma2Slice = Slice('ySigma',
                    lambda x: model.dataLikelihood(yKernel,k2_sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

yLengthscaleSlice = Slice('yLengthscale',
                    lambda x: model.dataLikelihood(yKernel,k2_lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)


kSigmaSlice = Slice('kSigma',
                    lambda x: prior.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-6,1e2).logpdf(x),
                    .2,5,logspace=True)

kLengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-1,1e2).logpdf(x),
                    .2,5,logspace=True)

k2SigmaSlice = Slice('kSigma',
                    lambda x: prior2.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-6,1e2).logpdf(x),
                    .2,5,logspace=True)

k2LengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior2.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-1,1e2).logpdf(x),
                    .2,5,logspace=True)

samples = []
freeze = Freezer(yKernel=yKernel,k1=k1,k2=k2,model=model)

In [ ]:
thin = 10
burnin = 0
nsample = 3000

for i in range(nsample):
    prior.sample(model,yKernel)
    prior2.sample(model,yKernel)
    
    yKernel.k1.sigma = ySigmaSlice.sample(yKernel.k1.sigma)
    yKernel.k2.sigma = ySigma2Slice.sample(yKernel.k2.sigma)
    yKernel.k2.lengthscale = yLengthscaleSlice.sample(yKernel.k2.lengthscale)
    
    k1.sigma = kSigmaSlice.sample(k1.sigma)
    k1.lengthscale = kLengthscaleSlice.sample(k1.lengthscale)
    
    k2.sigma = k2SigmaSlice.sample(k2.sigma)
    k2.lengthscale = k2LengthscaleSlice.sample(k2.lengthscale)

    if i % thin == 0 and i > burnin:
        f = freeze.freeze()
        del f['model']
        
        print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        print model.dataLikelihood(yKernel)
        printSample(f)
        print
        
        samples.append(freeze.freeze())

In [ ]:
samples = json.load(open("edgeEffect-edge-replicate.json"))

for s in samples:
    s['model']['beta'] = np.array(s['model']['beta'])

In [ ]:
plt.plot(x,np.array([s['model']['beta'][:,0] for s in samples]).T,c='k',alpha=.5);

In [ ]:
plt.plot(x,np.array([2*s['model']['beta'][:,1] for s in samples]).T,c='k',alpha=.5);

In [ ]:
plt.plot(x,scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]),yKernel.K(x),size=20).T);

In [ ]:
beta = samples[-1]['model']['beta']

plt.figure(figsize=(10,5))
for i,p in enumerate([0,1]):
    select = position==p
    
    plt.subplot(1,2,i+1)
    plt.plot(x,y[:,select],c='k',alpha=.4)
    plt.plot(x,y[:,select].mean(1),c='b',alpha=1)
    plt.plot(x,beta[:,0] + (1-2*p)*beta[:,1],c='r')
    
    plt.ylim(y.min(),y.max())

In [ ]:
beta = samples[-1]['model']['beta']

plt.figure(figsize=(10,10))
for i,p in enumerate([0,1]):
    select = position==p
    
    plt.subplot(2,2,i+1)
    plt.plot(x,y[:,select],c='k',alpha=.2)
    plt.plot(x,beta[:,0] + (1-2*p)*beta[:,1],c='r')
    
    plt.subplot(2,2,i+3)
    sample = scipy.stats.multivariate_normal.rvs(beta[:,0] + (1-2*p)*beta[:,1],yKernel.K(x),size=sum(select)).T
    plt.plot(x,sample,c='k',alpha=.2)
    plt.plot(x,beta[:,0] + (1-2*p)*beta[:,1],c='r')
    plt.ylim(y.min(),y.max())

In [ ]:
samplesCopy = [copy.deepcopy(s) for s in samples]
for s in samplesCopy:
    s['model']['beta'] = s['model']['beta'].tolist()
    
s = json.dumps(samplesCopy)

ofile = open("edgeEffect-edge-replicate.json",'w')
ofile.write(s)
ofile.close()